<header style="background:#00233C;padding-left:20pt;padding-right:20pt;padding-top:20pt;padding-bottom:10pt;"><img id="Teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 100px; height: auto; margin-top: 20pt;" align="right">
<p style="font-size:20px; color:#ffffff">UDW INNOVATION DAYS</p>
<p style="font-size:24px; color:#ffffff">Wrangling Data with Virtual DataFrames</p>
</header>

#### Install teradataml package
Note: You only need to run this once. The "!" allows you to run Linux script from the notebook cell. 

In [ ]:
#!pip install teradataml --user

#### Import Packages

In [3]:
# for managing connections
from teradataml import create_context, get_context, remove_context

# for setting configure options
from teradataml import configure

# to hide authentication strings
import getpass as gp

# DataFrames
from teradataml import DataFrame, in_schema, db_list_tables, execute_sql

import pandas as pd
import numpy as np

### Connection Variables

##### Set User and Password Variables

In [4]:
user = gp.getpass("User")

User ········


In [5]:
password = gp.getpass("Password")

Password ········


##### Set Connection Variables

In [6]:
host = 'UDWTest'
logmech = 'LDAP'
defaultDB = 'INOUDWTRAINING2024' 

##### Create Context
See the PythonBasics-1-ConnectingToVantage Notebook for more information about contexts and garbage collection.  

In [7]:
td_context = create_context(host = host, 
                            username= user, 
                            password = password, 
                            logmech='LDAP', 
                            database=defaultDB)

/home/tlugtu/conda/lib/python3.10/site-packages/teradataml/context/context.py:484: TeradataMlRuntimeWarning: Warning: Password is URL encoded.
  warnings.warn("Warning: Password is URL encoded.", category=TeradataMlRuntimeWarning)


### teradataml DataFrames (plots, species, surveys)
A teradataml DataFrame() can be constructed from a Vantage table, view, or query.

See help documentation for DataFrame - 

In [ ]:
help("teradataml.DataFrame")

#### Plots DataFrame and `.head()` Function
- "td_" will denote a teradataml dataframe
- "pd_" will denote a pandas dataframe

Use `in_schema()` function to pull a table from a different database.

`td_df = DataFrame(in_schema([other_db],[table_or_view]))`

`DataFrame.head(n)` function will show the first n records. Default n=10.

In [10]:
td_plots = DataFrame(in_schema(defaultDB, 'plots'))
td_plots.head(5)

plot_id,plot_type
3,Long-term Krat Exclosure
5,Rodent Exclosure
4,Control
2,Control
1,Spectab exclosure


#### teradataml DataFrames are In-Database

The actual plots data is in-database. The td_plots object is only holding SQL script locally in memory.

In [11]:
td_plots.show_query()

'select * from "INOUDWTRAINING2024"."plots"'

#### Species DataFrame

You can also use a SQL query with the function DataFrame.from_query() - 

`DataFrame.from_query([SELECT field1, field2 FROM other_db.table_or_view])`

In [12]:
td_species = DataFrame.from_query(f'select * from {defaultDB}.species')
td_species.head(5)

species_id,genus,species,taxa
AS,Ammodramus,savannarum,Bird
CB,Campylorhynchus,brunneicapillus,Bird
BA,Baiomys,taylori,Rodent
AH,Ammospermophilus,harrisi,Rodent
AB,Amphispiza,bilineata,Bird


In [13]:
td_species.show_query()

'select * from INOUDWTRAINING2024.species'

#### Surveys DataFrame
Note that default row display = 10 rows, but not in record order.

In [14]:
td_surveys = DataFrame(in_schema(defaultDB, 'surveys'))
td_surveys

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
20554,12,22,1992,4,DM,M,32.000,None
122,8,21,1977,22,DM,F,33.000,None
32300,7,22,2001,9,NL,None,34.000,95.000
30954,7,1,2000,21,PB,F,24.000,30.000
5220,1,24,1982,20,AB,None,None,None
11664,7,3,1986,19,OT,M,18.000,14.000
31383,9,30,2000,2,PB,M,27.000,33.000
24266,7,21,1996,11,OT,M,20.000,19.000
13437,9,27,1987,1,DS,M,49.000,132.000
13580,10,25,1987,7,RM,M,16.000,10.000


#### teradataml Display and Configure Options
Like with pandas, you can change the teradataml DataFrame diplay defaults using print_options properties. See all display and configure options below.

In [15]:
from teradataml import print_options
print_options()

Display Options
------------------
blob_length = 10
byte_encoding = base16
geometry_column_length = 30
max_rows = 10
precision = 3
print_sqlmr_query = False
suppress_vantage_runtime_warnings = False

Configure Options
------------------
byom_install_location = None
certificate_file = False
column_casesensitive_handler = False
cran_repositories = None
database_version = 17.20.03.26
default_varchar_size = 1024
indb_install_location = /var/opt/teradata/languages/sles12sp3/Python/
inline_plot = True
read_nos_function_mapping = read_nos
sandbox_container_id = None
temp_table_database = None
temp_view_database = None
ues_url = None
val_install_location = None
vantage_version = vantage1.3
write_nos_function_mapping = write_nos


##### Change max_rows
To change the number of rows displayed -

In [16]:
 display.max_rows = 100

### In-Database Data Wrangling with teradataml DataFrame
#### Sorting Data

In [17]:
td_surveys_sorted = td_surveys.sort(['plot_id','species_id'], ascending=[True,True])
td_surveys_sorted.head(5)

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
3,7,16,1977,2,DM,F,37.000,None
1,7,16,1977,2,NL,M,32.000,None
5,7,16,1977,3,DM,M,35.000,None
2,7,16,1977,3,NL,M,33.000,None
4,7,16,1977,7,DM,M,36.000,None


####  Subsetting Data
Extract the survey observations for the winter months of each year

In [18]:
# subset by selecting the winter months
td_surveys_winter = td_surveys[
    ((td_surveys.month == 1) | (td_surveys.month == 2) | (td_surveys.month == 12))]
td_surveys_winter.head(5)

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
488,12,11,1977,12,DM,M,37.000,41.000
591,2,18,1978,6,DM,F,38.000,None
528,1,8,1978,20,DS,M,51.000,122.000
448,12,10,1977,20,DS,M,51.000,116.000
446,12,10,1977,2,OT,None,None,None


#### View SQL behind the scenes - `show_query()`
Note that this query creates a temp - or volatile - table in the database.

In [19]:
td_surveys_winter.show_query()

'select * from "INOUDWTRAINING2024"."surveys" where "month" = 1 OR "month" = 2 OR "month" = 12'

#### Aggregations
Count the number of individuals by species observed in the winters. 

In [20]:
# Notice the default name of an aggregation field
td_surveys_winter_count = td_surveys_winter.groupby('species_id').count().select(['species_id', 'count_record_id'])
td_surveys_winter_count.head(5)

species_id,count_record_id
AH,3
CB,1
BA,2
AB,5
None,8


#### Layering DataFrame Methods

Note that each method applied on an object is adding another SQL query layer - i.e. layering subqueries. Note that they can cause the query to take longer or you might run out of spool space. Play with these to test your limits.

#### Renaming Columns 
Rename a Column by Creating a New One - `.assign()` - and Dropping the Old One - `.drop()`

In [21]:
# Create new field - .assign()
td_surveys_winter_count = td_surveys_winter_count.assign(count1 = td_surveys_winter_count.count_record_id)

# Drop old field - .drop()
td_surveys_winter_count = td_surveys_winter_count.drop(['count_record_id'], axis=1)

# View top 10 rows
td_surveys_winter_count.head()

species_id,count1
AH,3
CB,1
DM,96
DO,23
NL,9
OL,10
DS,20
BA,2
AB,5
None,8


#### Get a Total Count of Species

In [22]:
td_total_count = td_surveys_winter_count.select('count1').sum()
td_total_count

sum_count1
283


#### Conversion to pandas dataframe is supported by `DataFrame.to_pandas()`
Use pandas index to get the totals value

In [23]:
tot_cnt = td_total_count.to_pandas()['sum_count1'][0]
tot_cnt

283

#### Use the Total Count to Create a new Population Proportion Column Using `.assign()`

In [24]:
# we use the previously obtained count variable to derive the PROPORTION of individuals represented by
# each species, and assign the result to a new 'pop_prop' column.
td_surveys_winter_count = td_surveys_winter_count.assign(pop_prop = td_surveys_winter_count.count1 / float(tot_cnt))
td_surveys_winter_count

species_id,count1,pop_prop
OT,11,0.038869257950530034
SF,1,0.0035335689045936395
AH,3,0.01060070671378092
CB,1,0.0035335689045936395
DO,23,0.0812720848056537
None,8,0.028268551236749116
PM,7,0.024734982332155476
OL,10,0.0353356890459364
DS,20,0.0706713780918728
BA,2,0.007067137809187279


In [25]:
# view the sql query
td_surveys_winter_count.show_query()

'select species_id AS species_id, count1 AS count1, count1 / CAST(CAST(283.0 as FLOAT) AS FLOAT) AS pop_prop from (select species_id,count1 from (select species_id AS species_id, count_record_id AS count_record_id, count_record_id AS count1 from (select species_id,count_record_id from (select "species_id", count(record_id) AS count_record_id, count("month") AS count_month, count("day") AS count_day, count("year") AS count_year, count(plot_id) AS count_plot_id, count(sex) AS count_sex, count(hindfoot_length) AS count_hindfoot_length, count(weight) AS count_weight from (select * from "INOUDWTRAINING2024"."surveys" where "month" = 1 OR "month" = 2 OR "month" = 12) as temp_table group by "species_id") as temp_table) as temp_table) as temp_table) as temp_table'

#### Join DataFrames
Add the count and proportion to the species table

In [26]:
# Inner Join between td_surveys_1990w_count and td_species
td_winter_inner_join = td_surveys_winter_count.join(td_species, on = 'species_id', how = "inner", lsuffix = 't1', rsuffix = 't2')

# Note first two columns are duplicates
td_winter_inner_join

/tmp/ipykernel_2899/3336986459.py:2: DeprecationWarning: 
Behaviour of the argument(s) "['lsuffix', 'rsuffix']" will change in future.
Use argument(s) "['lprefix', 'rprefix']" instead.
  td_winter_inner_join = td_surveys_winter_count.join(td_species, on = 'species_id', how = "inner", lsuffix = 't1', rsuffix = 't2')


t1_species_id,t2_species_id,count1,pop_prop,genus,species,taxa
PG,PG,1,0.0035335689045936395,Pooecetes,gramineus,Bird
OT,OT,11,0.038869257950530034,Onychomys,torridus,Rodent
PE,PE,4,0.014134275618374558,Peromyscus,eremicus,Rodent
PM,PM,7,0.024734982332155476,Peromyscus,maniculatus,Rodent
BA,BA,2,0.007067137809187279,Baiomys,taylori,Rodent
PF,PF,9,0.03180212014134275,Perognathus,flavus,Rodent
PC,PC,1,0.0035335689045936395,Pipilo,chlorurus,Bird
AB,AB,5,0.0176678445229682,Amphispiza,bilineata,Bird
PP,PP,10,0.0353356890459364,Chaetodipus,penicillatus,Rodent
DM,DM,96,0.3392226148409894,Dipodomys,merriami,Rodent


#### Rename by Creating New and Dropping Old Columns 

In [27]:
td_winter_inner_join = td_winter_inner_join.assign(species_id = td_winter_inner_join.t1_species_id)
td_winter_inner_join = td_winter_inner_join.drop(['t1_species_id', 't2_species_id'], axis = 1)
td_winter_inner_join.sort(["species_id"])

count1,pop_prop,genus,species,taxa,species_id
5,0.0176678445229682,Amphispiza,bilineata,Bird,AB
3,0.01060070671378092,Ammospermophilus,harrisi,Rodent,AH
2,0.007067137809187279,Baiomys,taylori,Rodent,BA
1,0.0035335689045936395,Campylorhynchus,brunneicapillus,Bird,CB
96,0.3392226148409894,Dipodomys,merriami,Rodent,DM
23,0.0812720848056537,Dipodomys,ordii,Rodent,DO
20,0.0706713780918728,Dipodomys,spectabilis,Rodent,DS
9,0.03180212014134275,Neotoma,albigula,Rodent,NL
10,0.0353356890459364,Onychomys,leucogaster,Rodent,OL
11,0.038869257950530034,Onychomys,torridus,Rodent,OT


#### The Volatile View
The DataFrame sql is saved as a volatile (temporary) View, which is referenced below - "ml__join__[some number]". The previous manipulation, which dropped a column, is now filtering that view.

When you remove_context() at the end of your notebook, these volatile views will be dropped automatically.

In [28]:
# View the query
td_winter_inner_join.show_query()

'select count1,pop_prop,genus,species,taxa,species_id from (select t1_species_id AS t1_species_id, t2_species_id AS t2_species_id, count1 AS count1, pop_prop AS pop_prop, genus AS genus, species AS species, taxa AS taxa, t1_species_id AS species_id from "INOUDWTRAINING2024"."ml__join__1721214745211017") as temp_table'

#### Other Helpful Properties of teradataml DataFrames

##### You can get the `shape` property.

In [29]:
td_winter_inner_join.shape

(23, 6)

##### Get Column Names

In [30]:
td_winter_inner_join.columns

['count1', 'pop_prop', 'genus', 'species', 'taxa', 'species_id']

##### You can see the SQL datatypes by using `.tdtypes` (Teradata Data Types)

In [31]:
td_winter_inner_join.tdtypes

COLUMN NAME,TYPE
count1,INTEGER()
pop_prop,FLOAT()
genus,"VARCHAR(length=64000, charset='LATIN')"
species,"VARCHAR(length=64000, charset='LATIN')"
taxa,"VARCHAR(length=64000, charset='LATIN')"
species_id,"VARCHAR(length=64000, charset='LATIN')"


##### You can also see the corresponding pandas datatypes if you were to convert to pandas using `.dtypes`

In [32]:
td_winter_inner_join.dtypes

COLUMN NAME,TYPE
count1,int
pop_prop,float
genus,str
species,str
taxa,str
species_id,str


#### Save Volatile DataFrame as Permanent Table Using `to_sql()`

In [33]:
td_winter_inner_join.to_sql("perm_table_from_join", if_exists="replace")

#### Retrieve Permanent Table to a New teradataml Dataframe

In [34]:
td_perm_table = DataFrame('perm_table_from_join')
td_perm_table.head(5)

count1,pop_prop,genus,species,taxa,species_id
1,0.0035335689045936395,Sigmodon,hispidus,Rodent,SH
1,0.0035335689045936395,Sigmodon,fulviventer,Rodent,SF
1,0.0035335689045936395,Pipilo,chlorurus,Bird,PC
1,0.0035335689045936395,Reithrodontomys,fulvescens,Rodent,RF
1,0.0035335689045936395,Pooecetes,gramineus,Bird,PG


In [35]:
td_perm_table.show_query()

'select * from "perm_table_from_join"'

#### Left Join Example
Observe one species with a `None` species_id, which is not present in td_winter_inner_join table

In [36]:
# Left Join
td_winter_left_join = td_surveys_winter_count.join(td_species, on = 'species_id', how = "left", lsuffix = 't1', rsuffix = 't2')

# New Column with assign() to rename
td_winter_left_join = td_winter_left_join.assign(species_id = td_winter_left_join.t1_species_id)

# Drop old duplicate columns
td_winter_left_join = td_winter_left_join.drop(['t1_species_id', 't2_species_id'], axis = 1)

# View Shape 
print(td_winter_left_join.shape)

# View data
td_winter_left_join.sort(["species_id"]) 

/tmp/ipykernel_2899/2795168631.py:2: DeprecationWarning: 
Behaviour of the argument(s) "['lsuffix', 'rsuffix']" will change in future.
Use argument(s) "['lprefix', 'rprefix']" instead.
  td_winter_left_join = td_surveys_winter_count.join(td_species, on = 'species_id', how = "left", lsuffix = 't1', rsuffix = 't2')


(24, 6)


count1,pop_prop,genus,species,taxa,species_id
8,0.028268551236749116,None,None,None,None
5,0.0176678445229682,Amphispiza,bilineata,Bird,AB
3,0.01060070671378092,Ammospermophilus,harrisi,Rodent,AH
2,0.007067137809187279,Baiomys,taylori,Rodent,BA
1,0.0035335689045936395,Campylorhynchus,brunneicapillus,Bird,CB
96,0.3392226148409894,Dipodomys,merriami,Rodent,DM
23,0.0812720848056537,Dipodomys,ordii,Rodent,DO
20,0.0706713780918728,Dipodomys,spectabilis,Rodent,DS
9,0.03180212014134275,Neotoma,albigula,Rodent,NL
10,0.0353356890459364,Onychomys,leucogaster,Rodent,OL


#### Inner Join Sum of Population Proportion < 1.0
This point is also proved by comparing the sum of population proportions between the Inner Join and the Left Join.

In [37]:
td_winter_inner_join.sum()

sum_count1,sum_pop_prop
275,0.9717314487632509


#### Left Join Sum of Population Proportion = 1.0

In [38]:
td_winter_left_join.sum()

sum_count1,sum_pop_prop
283,1.0000000000000002


### Remove context
This best practice performs garbage collection for the volatile tables and views that were created during this session.

In [39]:
remove_context()

True

<span style="font-size:16px;">For online documentation on Teradata Vantage analytic functions, refer to the [Teradata Developer Portal](https://docs.teradata.com/) and search for phrases "Python User Guide" and "Python Function Reference".</span>